<a href="https://colab.research.google.com/github/lazarovps/Data-Science/blob/main/covid_PE_microdados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ANÁLISE DO PERFIL  DOS RECUPERADOS E DOS MORTOS POR COVID-19 EM PERNAMBUCO**.

# OBJETIVO

O objetivo dessa análise é de verificar se há e quais são as diferenças entre aqueles que se recuperaram e aqueles que morreram pelo COVID-19 em Pernambuco. Para isso, analisar-se-á os dados disponibilizados pela SEPLAG-PE sobre os casos registrados de covid no estado, a data da coleta das informações foi 05/08/2021 mas, provavelmente há a defasagem de alguns dias. A base de dados será dividida em recuperados e obitos e serão observados fatores demográficos, frequência determinados sintomas e comorbidades. Através de testes estatísticos, pretende-se observar se há diferença entre as proporções de determinados fatores entre os recuperados e que vieram à óbito. 

# IMPORTAÇÃO DAS BIBLIOTECAS NECESSÁRIAS E LOAD DOS DADOS

In [ ]:
from google.colab import drive 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, accuracy_score, confusion_matrix
from warnings import simplefilter

simplefilter(action='ignore', category=FutureWarning)
#configurando o diretório de trabalho
drive.mount('/content/drive')
%cd /content/drive/My Drive/Python

#Dados da seplag estadual https://dados.seplag.pe.gov.br/apps/corona_dados.html


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Python


In [ ]:
base = pd.read_csv("covid_pe.csv", sep = ";")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# TRATAMENTO DOS DADOS

In [ ]:
base.shape

(1745924, 15)

A base de dados importada possui aproximadamente 1,7 milhão de linhas e 15 colunas, cada linha representa, em teoria, um individuo, e cada coluna seria uma informação sobre esse indivíduo. Ao observarmos as primeira 3 linhas desses dados, podemos ter uma noção de quais são as informações disponibilizadas.

In [ ]:
base.head(3)

,dt_notificacao,Sexo,raca,municipio,dt_primeiros_sintomas,sintomas,comorbidades,hospitalizacao,classe,Resultado,evolucao,dt_obito,cd_municipio,faixa_etaria,tipo
0,2020-04-23,FEMININO,NaN,RECIFE,2020-04-13,FEBRE DOR DE GARGANTA CEFALEIA PERDA DE OLFA...,NaN,NAO,EM INVESTIGACAO,AGUARDANDO RESULTADO,EM TRATAMENTO DOMICILIAR,NaN,261160,20-29 anos,grave
1,2020-04-20,FEMININO,NaN,RECIFE,2020-03-06,DOR DE GARGANTA FEBRE,NaN,NaN,EM INVESTIGACAO,NaN,NaN,NaN,261160,30-39 anos,leve
2,2020-04-23,FEMININO,NaN,RECIFE,2020-04-18,DOR DE GARGANTA OUTROS,NaN,NaN,EM INVESTIGACAO,NaN,NaN,NaN,261160,60-69 anos,leve


Antes de dividir a base de dados em recuperados e óbitos, é interessante verificar,manipular algumas colunas, essa é a fase que nos faz melhor entender nossos dados.

Um primeiro passo é a verificação de valores nulos, a contagem das células nulas é muito importante para termos uma noção da qualidade dos nossos dados, infelizmente há muitas células nulas nesses dados, mas há muita coisa que podemos analisar.

Descartando algumas colunas que não serão utilizadas na análise, ficamos com apenas 7 colunas:

In [ ]:
base = base.drop(['dt_notificacao','municipio','dt_primeiros_sintomas','hospitalizacao','classe','Resultado', 'dt_obito','cd_municipio'], axis = 1)

In [ ]:
base.head(5)

,Sexo,raca,sintomas,comorbidades,evolucao,faixa_etaria,tipo
0,FEMININO,NaN,FEBRE DOR DE GARGANTA CEFALEIA PERDA DE OLFA...,NaN,EM TRATAMENTO DOMICILIAR,20-29 anos,grave
1,FEMININO,NaN,DOR DE GARGANTA FEBRE,NaN,NaN,30-39 anos,leve
2,FEMININO,NaN,DOR DE GARGANTA OUTROS,NaN,NaN,60-69 anos,leve
3,MASCULINO,NaN,FEBRE TOSSE,NaN,NaN,30-39 anos,leve
4,FEMININO,NaN,NaN,NaN,EM TRATAMENTO DOMICILIAR,40-49 anos,grave


Realizados os primeiros descartes, é interessante verificar quantos valores distintos temos por coluna, que nos dirá a qualidade das informações que estamos analisando.

In [ ]:
base.isnull().sum()

Sexo                  0
raca              15181
sintomas        1657490
comorbidades    1572399
evolucao         202126
faixa_etaria        695
tipo                  0
dtype: int64

Percebe-se que há muitas céluas nulas em quase todas as colunas, o ideal seria o mínimo possível, mas ainda podemos fazer uma análise robusta.

Verificando o tipo das colunas para então começar a manipulação de dados:

In [ ]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1745924 entries, 0 to 1745923
Data columns (total 7 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   Sexo          object
 1   raca          object
 2   sintomas      object
 3   comorbidades  object
 4   evolucao      object
 5   faixa_etaria  object
 6   tipo          object
dtypes: object(7)
memory usage: 93.2+ MB


In [ ]:
#Começando por 'Sexo'
base['Sexo'].value_counts()

FEMININO      939045
MASCULINO     715953
Masculino      46338
Feminino       44091
INDEFINIDO       493
Ignorado           4
Name: Sexo, dtype: int64

O Python é sensitive case, então, FEMININO é diferente de Feminino, então padronizemos para todo o texto ficar minúsculo.

In [ ]:
base['Sexo'] = base['Sexo'].str.lower()

In [ ]:
base['raca'].value_counts()

PARDA       861028
BRANCA      373477
IGNORADO    345277
AMARELA      84172
PRETA        59761
INDIGENA      7028
Name: raca, dtype: int64

A variável raça aparentemente está sem problemas.

Agora partindo para a coluna de sintomas, que possui incríveis 6 mil valores distintos!!

In [ ]:
#CLASSES DOS SINTOMAS:

#dispneia
#desconforto respiratorio
#aperto toracico
#saturacao 02 < 95
#febre
#coriza
#congestao nasal
#cansacofadiga
#mialgia
#cefaleia
#diarreia
#dor de garganta

In [ ]:
base['sintomas'].value_counts()

FEBRE TOSSE DISPNEIA SATURACAO O2 < 95                                                                                                       6391
FEBRE TOSSE DISPNEIA                                                                                                                         4982
FEBRE TOSSE DISPNEIA DESCONFORTO RESPIRATORIO  APERTO TORACICO SATURACAO O2 < 95                                                             4472
TOSSE DISPNEIA SATURACAO O2 < 95                                                                                                             2185
FEBRE TOSSE                                                                                                                                  2149
                                                                                                                                             ... 
VOMITO FEBRE TOSSE DESCONFORTO RESPIRATORIO  APERTO TORACICO SATURACAO O2 < 95                                              

Após fazer uma breve análise de alguns valores dessa coluna, podemos constatar que na verdade essa informação provavelmente era coletada em formada de caixa de seleção, então o agente de saúde marcava várias opções e quando essa informação foi para o sistema cada seleção foi convertida em texto, para sabermos de fato quantas pessoas tiveram cada tipo de sintoma, é necessário separá-los por colunas.

As "classes" de sintomas que aparentemente foram mais presentes são:

* Alteração e/ou perda de olfato e/ou paladar
* aperto torácico
* cansaço/fadiga
* cefaleia
* congestão nasal
* coriza
* diarreia
* desconforto respiratório
* dispnéia
* dor de garganta
* edema mãos e/ou pés
* febre
* mialgia
* naúsea
* saturação de 0² < 95


Transformando a coluna 'sintomas' em diversas colunas:

In [ ]:
base['alteracao_olfato_paladar'] = base['sintomas'].str.extract('(ALTERACAOPERDA DE OLFATO EOU PALADAR)', expand = False).str.strip()
base['aperto_toracico'] = base['sintomas'].str.extract('(APERTO TORACICO)', expand = False).str.strip()
base['cansaco'] = base['sintomas'].str.extract('(CANSACOFADIGA)', expand = False).str.strip()
base['cefaleia'] = base['sintomas'].str.extract('(CEFALEIA)', expand = False).str.strip()
base['congestao_nasal'] = base['sintomas'].str.extract('(CONGESTAO NASAL)', expand = False).str.strip()
base['coriza'] = base['sintomas'].str.extract('(CORIZA)', expand = False).str.strip()
base['diarreia'] = base['sintomas'].str.extract('(DIARREIA)', expand = False).str.strip()
base['descon_resp'] = base['sintomas'].str.extract('(DESCONFORTO RESPIRATORIO)', expand = False).str.strip()
base['dispneia'] = base['sintomas'].str.extract('(DISPNEIA)', expand = False).str.strip()
base['dor_de_garganta'] = base['sintomas'].str.extract('(DOR DE GARGANTA)', expand = False).str.strip()
base['edema'] = base['sintomas'].str.extract('(EDEMA MAOSPES)', expand = False).str.strip()
base['febre'] = base['sintomas'].str.extract('(FEBRE)', expand = False).str.strip()
base['mialgia'] = base['sintomas'].str.extract('(MIALGIA)', expand = False).str.strip()
base['nausea'] = base['sintomas'].str.extract('(NAUSEA)', expand = False).str.strip()
base['saturacao<95'] = base['sintomas'].str.extract('(SATURACAO O2 < 95)', expand = False).str.strip()
base.iloc[:,-15:] = np.where(base.iloc[:,-15:].isna(),0,1)

In [ ]:
base.iloc[:,-15:].sum()

alteracao_olfato_paladar     6743.0
aperto_toracico             32062.0
cansaco                     10984.0
cefaleia                    11215.0
congestao_nasal              7401.0
coriza                       7407.0
diarreia                     8921.0
descon_resp                 32377.0
dispneia                    63548.0
dor_de_garganta             14188.0
edema                         504.0
febre                       59060.0
mialgia                     10526.0
nausea                       2444.0
saturacao<95                50112.0
dtype: float64

In [ ]:
base['comorbidades'].value_counts()

DOENCAS CARDIACAS CRONICAS                       39120
DIABETES                                         27075
GESTANTE                                         22381
DOENCAS RESPIRATORIAS CRONICAS DESCOMPENSADAS    18516
DOENCAS CARDIACAS OU VASCULARES                  10024
                                                 ...  
SECRECAO CONJUNTIVAL                                 1
262000381994                                         1
DESORIENTACAO                                        1
262000407550                                         1
262000385612                                         1
Name: comorbidades, Length: 974, dtype: int64

Semelhante à coluna 'sintomas', a coluna 'comorbidades' necessita de manipulação, as classes aparentemente são:

* cefaleia e fadiga (será descartada, já que são sintomas)
* diabetes
* doenças cardíacas ou vasculares
* doenças hepáticas crônicas
* doenças renais crônicas
* doenças respiratórias
* gestação
* imunosupressão
* portador de doenças cromossômicas
* sobrepeso/obesidade


In [ ]:
base['diabetes'] = base['comorbidades'].str.extract('(DIABETES)', expand = False).str.strip()
base['card_or_vasc'] = base['comorbidades'].str.extract('(DOENCAS CARDIACAS OU VASCULARES)', expand = False).str.strip()
base['hepatic_cronic'] = base['comorbidades'].str.extract('(DOENCA HEPATICA CRONICA)', expand = False).str.strip()
base['renais_cronic'] = base['comorbidades'].str.extract('(DOENCAS RENAIS CRONICAS)', expand = False).str.strip()
base['resp_cronic'] = base['comorbidades'].str.extract('(DOENCAS RESPIRATORIAS CRONICAS)', expand = False).str.strip()
base['gestacao'] = base['comorbidades'].str.extract('(GESTACAO)', expand = False).str.strip()
base['imunossupressao'] = base['comorbidades'].str.extract('(IMUNOSSUPRESSAO)', expand = False).str.strip()
base['cromossomicas'] = base['comorbidades'].str.extract('(PORTADOR DE DOENCAS CROMOSSOMICAS)', expand = False).str.strip()
base['obesidade'] = base['comorbidades'].str.extract('(SOBREPESOOBESIDADE)', expand = False).str.strip()
base.iloc[:,-9:] = np.where(base.iloc[:,-9:].isna(),0,1)

In [ ]:
base.iloc[:,-9:].sum()

diabetes           53146.0
card_or_vasc       22224.0
hepatic_cronic       687.0
renais_cronic       6379.0
resp_cronic        27102.0
gestacao             345.0
imunossupressao     8071.0
cromossomicas       2585.0
obesidade           4548.0
dtype: float64

In [ ]:
#Consultando novamente as colunas do nosso dataframe
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1745924 entries, 0 to 1745923
Data columns (total 31 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   Sexo                      object
 1   raca                      object
 2   sintomas                  object
 3   comorbidades              object
 4   evolucao                  object
 5   faixa_etaria              object
 6   tipo                      object
 7   alteracao_olfato_paladar  object
 8   aperto_toracico           object
 9   cansaco                   object
 10  cefaleia                  object
 11  congestao_nasal           object
 12  coriza                    object
 13  diarreia                  object
 14  descon_resp               object
 15  dispneia                  object
 16  dor_de_garganta           object
 17  edema                     object
 18  febre                     object
 19  mialgia                   object
 20  nausea                    object
 21  saturaca

In [ ]:
base['evolucao'].value_counts()

RECUPERADO                       1461130
OBITO                              26734
IGNORADO                           24685
INTERNADO LEITO DE ISOLAMENTO      16699
EM TRATAMENTO DOMICILIAR           13411
INTERNADO UTI                       1139
Name: evolucao, dtype: int64

In [ ]:
base['faixa_etaria'].value_counts()

30-39 anos    400323
20-29 anos    334504
40-49 anos    325849
50-59 anos    234151
60-69 anos    134135
10-19 anos    122319
0-9 anos       84957
70-79 anos     69698
80+ anos       39293
Name: faixa_etaria, dtype: int64

In [ ]:
base['tipo'].value_counts()

leve     1654669
grave      91255
Name: tipo, dtype: int64

In [ ]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1745924 entries, 0 to 1745923
Data columns (total 31 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   Sexo                      object
 1   raca                      object
 2   sintomas                  object
 3   comorbidades              object
 4   evolucao                  object
 5   faixa_etaria              object
 6   tipo                      object
 7   alteracao_olfato_paladar  object
 8   aperto_toracico           object
 9   cansaco                   object
 10  cefaleia                  object
 11  congestao_nasal           object
 12  coriza                    object
 13  diarreia                  object
 14  descon_resp               object
 15  dispneia                  object
 16  dor_de_garganta           object
 17  edema                     object
 18  febre                     object
 19  mialgia                   object
 20  nausea                    object
 21  saturaca

# COMPARAÇÃO ENTRE RECUPERADOS E ÓBITOS

Quando criamos novas colunas com base em algumas já existentes, acabamos atribuindo valores às células que estavam sem informação (por exemplo, a coluna sintomas tinha relativamente poucos valores não-nulos e todas as colunas que foram derivadas dessas estão sem valores nulo, significa então que "criamos" informação), como o objetivo desse estudo é de verificar a proporção de algumas características, é necessário descartar essas informações que criamos, então, para cada característica analisada, será criado um subgrupo de valores não-nulos de sua coluna inicial.

In [ ]:
recuperados = base[base['evolucao'] == 'RECUPERADO']
obitos = base[base['evolucao'] == 'OBITO']

In [ ]:
recuperados.shape, obitos.shape

((1461130, 31), (26734, 31))

Ao separarmos em recuperados e óbitos, vemos que os números são bem discrepantes, então sempre olharemos para as proporções, que são medidas relativas.

Vendo a proporção dos sexos para cada um dos grupos temos:

In [ ]:
print("Proporção dos sexos de 'recuperados' \n\n",recuperados['Sexo'].value_counts(normalize = True), end= '\n\n')
print("Proporção dos sexos de 'obitos' \n\n",obitos['Sexo'].value_counts(normalize= True))

Proporção dos sexos de 'recuperados' 

 feminino      0.566352
masculino     0.433331
indefinido    0.000317
Name: Sexo, dtype: float64

Proporção dos sexos de 'obitos' 

 masculino    0.535236
feminino     0.464764
Name: Sexo, dtype: float64


Interessante observar que enquanto no grupo 'recuperados' as mulheres representam aproximadamente 57 %, no grupo 'obitos' esse percentual cai para 46%.
Analogamente, o percentual de homens é de 10%p.p maior no sgrupo 'obitos'´do que no grupo 'recuperados'.



Para vermos essas proporções em relação à cor/raça, temos que dropar os valores que estão como 'IGNORADO'. Realizando essa operação nesses grupos que criamos:

In [ ]:
recuperados_raca = recuperados[recuperados['raca'] != 'IGNORADO']
obitos_raca = obitos[obitos['raca'] != 'IGNORADO']

In [ ]:
print("Proporção da cor/raça de 'recuperados' \n\n",recuperados_raca['raca'].value_counts(normalize = True), end= '\n\n')
print("Proporção da cor/raça de 'obitos' \n\n",obitos_raca['raca'].value_counts(normalize= True))

Proporção da cor/raça de 'recuperados' 

 PARDA       0.616816
BRANCA      0.271593
AMARELA     0.062795
PRETA       0.043692
INDIGENA    0.005104
Name: raca, dtype: float64

Proporção da cor/raça de 'obitos' 

 PARDA       0.688964
BRANCA      0.242224
PRETA       0.048576
AMARELA     0.018784
INDIGENA    0.001452
Name: raca, dtype: float64


Quando ordenamos decrescentemente as proporções, as única diferenças são nas posições 3 e 4, que no grupo 'recuperados' são as cores/raças 'amarelas' e 'pretas' e no grupo 'obitos' são 'pretas' e 'amarelas', respectivamente..

Em ambos os grupos há uma maior porcentagem de pessoas consideradas pardas, mas, o valor no grupo 'obitos' é 8%p.p em relação ao grupo 'recuperados'.

In [ ]:
print("Proporção das idades de 'recuperados'\n\n",recuperados_raca['faixa_etaria'].value_counts(normalize = True), end= '\n\n')
print("Proporção das idades de 'obitos'\n\n",obitos_raca['faixa_etaria'].value_counts(normalize= True))

Proporção das idades de 'recuperados'

 30-39 anos    0.233494
20-29 anos    0.198752
40-49 anos    0.188438
50-59 anos    0.133908
60-69 anos    0.073241
10-19 anos    0.071186
0-9 anos      0.047119
70-79 anos    0.035731
80+ anos      0.018131
Name: faixa_etaria, dtype: float64

Proporção das idades de 'obitos'

 80+ anos      0.250475
70-79 anos    0.243520
60-69 anos    0.209521
50-59 anos    0.147183
40-49 anos    0.079834
30-39 anos    0.042423
20-29 anos    0.015163
0-9 anos      0.007128
10-19 anos    0.004752
Name: faixa_etaria, dtype: float64


Olhando a proporção por faixa etária, vemos que a maior parte (cerca de 62%) do grupo dos recuperados possuem entre 20 e 49 anos. Já quando olhamos para o grupo de óbitos, temos a predominância de pessoas com 50 anos ou mais, representando aproximadamente 85 % das mortes.

Pode parecer óbvio à primeiro momento, mas é interessante observar a gravidades dos casos de cada grupo, já que podemos acabar descobrindo se uma quantidade razoável de casos evoluiu a ponto de levar o paciente à óbito.

In [ ]:
print("Gravidade do covid nos recuperados \n\n",recuperados_raca['tipo'].value_counts(normalize = True), end= '\n\n')
print("Gravidade do covid nos mortos\n\n",obitos_raca['tipo'].value_counts(normalize= True))

Gravidade do covid nos recuperados 

 leve     0.971656
grave    0.028344
Name: tipo, dtype: float64

Gravidade do covid nos mortos

 grave    1.0
Name: tipo, dtype: float64


Ao olharmos para os recuperados, constatamos que aproximadamente 3 % desses estavam em estado grave. 

Já em relação aos óbitos, talvez, quando houve a evolução do caso o tipo registrado mudou de leve para grave, então logicamente só morreriam pacientes do tipo grave.


Para fazermos uma análise semelhante com os sintomas, é necessário criar novos subgrupos para filtrarmos apenas as observações que não estavam com a coluna 'sintomas' nula.

In [ ]:
recuperados_sintomas = recuperados[recuperados['sintomas'].notna()].iloc[:,7:22]
obitos_sintomas = obitos[obitos['sintomas'].notna()].iloc[:,7:22]

In [ ]:
recuperados_sintomas.shape, obitos_sintomas.shape

((38030, 15), (25947, 15))

Nesse caso pegamos apenas as colunas que foram derivadas da coluna 'sintomas' (15 colunas, uma pra cada sintoma identificado). 
Pelo fato de termos criado colunas binárias, a média de cada uma das colunas já irá representar a porcentagem de manifestação de sintomas daqueles individuos que tiveram algum registro de sintomas.

Criando um objeto para a comparação das médias:

In [ ]:
comparacao_sint = pd.DataFrame({'obitos':obitos_sintomas.mean(),'recuperados':recuperados_sintomas.mean()})
comparacao_sint['diferenca'] = comparacao_sint['obitos'] - comparacao_sint['recuperados']
print("Diferença na proporção de sintomas no grupo de óbitos e no de recuperados\n")
print(comparacao_sint.sort_values(by = 'diferenca', ascending = False))

Diferença na proporção de sintomas no grupo de óbitos e no de recuperados

                            obitos  recuperados  diferenca
saturacao<95              0.710256     0.546227   0.164029
dispneia                  0.814391     0.671996   0.142395
descon_resp               0.441939     0.330660   0.111279
aperto_toracico           0.440321     0.330660   0.109661
edema                     0.008093     0.003182   0.004912
cansaco                   0.130266     0.130108   0.000158
nausea                    0.021351     0.032422  -0.011071
diarreia                  0.085135     0.117881  -0.032746
congestao_nasal           0.050834     0.090981  -0.040146
coriza                    0.050834     0.090981  -0.040146
mialgia                   0.095194     0.147699  -0.052505
alteracao_olfato_paladar  0.050295     0.110597  -0.060302
cefaleia                  0.079778     0.166290  -0.086512
dor_de_garganta           0.089028     0.189666  -0.100638
febre                     0.595676     0

Além da média de manifestação de cada um dos sintomas, é interessante observar a diferença entre os grupos, os sintomas que mais devemos ficar mais atentos são aqueles que a diferença foi positiva, já que significa que o percentual de manifestação desses sintomas é maior naqueles que morreram. Os sintomas que possuem maior diferença são:


* Saturação de 0² < 95 (16,40%)
* Dispnéia (14,24%)
* Desconforto respiratório (11,12%)
* Aperto torácico (10,97%)

(Talvez esses sintomas sejam redundantes, por não ter conhecimento na área, considerei como sintomas diferentes.)



Para verificar se essas  diferenças de proporções são relevantes ou se foram ao acaso, podemos realizar um teste de igualdade de proporções, onde as hipóteses são:

*   H0: (Prop.O - Prop.R) <= 0 (Proporção em 'obitos' menos Proporção em 'recuperados' menor ou igual a zero)
*   H1: (Prop.O - Prop.R) > 0 (Proporção em 'obitos' menos Proporção em 'recuperados' maior que zero)

Para um teste unilateral à direita (queremos verificar quais proporções de sintomas nos óbitos são maiores do que nos recuperados), com a = 0,01, temos que o Zcrítico é de 2,327. Então nossa regra de decisão:

*  Rejeitamos H0  se Zcalc > 2,327
*  Caso contrário, não rejeitar H0


Criando uma função para realização do teste teremos:

In [ ]:
def proportions_test(df1,df2, col):
  col = col
  df1 = df1
  df2 = df2
  x1 = len(df1[df1[col] == 1])
  x2 = len(df2[df2[col] == 1])
  n1 = len(df1)
  n2 = len(df2)
  p1 = x1/n1
  p2 = x2/n2
  pmean = (x1+x2)/(n1+n2)
  zcalc = (p1 - p2)/((pmean*(1-pmean)*((1/n1)+(1/n2)))**0.5)
  return zcalc

In [ ]:
#Realizando o teste para cada coluna dos grupos teremos
colunas = recuperados_sintomas.columns
for col in colunas:
  teste_prop = proportions_test(obitos_sintomas,recuperados_sintomas,col)
  print("O teste para diferença das proporções da coluna '{}' foi de {}".format(col,round(teste_prop,2)))

O teste para diferença das proporções da coluna 'alteracao_olfato_paladar' foi de -26.69
O teste para diferença das proporções da coluna 'aperto_toracico' foi de 28.13
O teste para diferença das proporções da coluna 'cansaco' foi de 0.06
O teste para diferença das proporções da coluna 'cefaleia' foi de -31.82
O teste para diferença das proporções da coluna 'congestao_nasal' foi de -18.96
O teste para diferença das proporções da coluna 'coriza' foi de -18.96
O teste para diferença das proporções da coluna 'diarreia' foi de -13.29
O teste para diferença das proporções da coluna 'descon_resp' foi de 28.53
O teste para diferença das proporções da coluna 'dispneia' foi de 39.82
O teste para diferença das proporções da coluna 'dor_de_garganta' foi de -35.11
O teste para diferença das proporções da coluna 'edema' foi de 8.5
O teste para diferença das proporções da coluna 'febre' foi de -30.59
O teste para diferença das proporções da coluna 'mialgia' foi de -19.62
O teste para diferença das pr

Olhando o resultado, vemos que 5 do 15 sintomas tiveram o valor maior que o definido, ou seja, esses 5 sintomas ('saturação<95','dispneia','descon_resp','aperto_toracico' e 'edema') têm uma proporção de manisfestação maior nos que morreram do que nos recuperados.

Além disso, podemos também verificar o intervalo em quê essa diferença entre as proporções estaria, para isso criamos novamente uma outra função.

In [ ]:
def intervalo_diferenca(df1,df2,col):
  col = col
  df1 = df1
  df2 = df2
  x1 = len(df1[df1[col] == 1])
  x2 = len(df2[df2[col] == 1])
  n1 = len(df1)
  n2 = len(df2)
  p1 = x1/n1
  p2 = x2/n2
  dif = (p1 - p2)
  inf = round((dif - 2.576*((((p1*(1-p1))/n1)+((p2*(1-p2))/n2))**0.5)),4)
  sup = round((dif + 2.576*((((p1*(1-p1))/n1)+((p2*(1-p2))/n2))**0.5)),4)
  return (inf,sup)

In [ ]:
#Encontrando os intervalos de diferença para cada coluna
colunas = recuperados_sintomas.columns
for col in colunas:
  teste_prop = intervalo_diferenca(obitos_sintomas,recuperados_sintomas,col)
  print("O intervalo para a diferença das proporções da variável '{}' é entre {}".format(col,teste_prop))

O intervalo para a diferença das proporções da variável 'alteracao_olfato_paladar' é entre (-0.0657, -0.0549)
O intervalo para a diferença das proporções da variável 'aperto_toracico' é entre (0.0996, 0.1197)
O intervalo para a diferença das proporções da variável 'cansaco' é entre (-0.0068, 0.0071)
O intervalo para a diferença das proporções da variável 'cefaleia' é entre (-0.0931, -0.08)
O intervalo para a diferença das proporções da variável 'congestao_nasal' é entre (-0.0453, -0.035)
O intervalo para a diferença das proporções da variável 'coriza' é entre (-0.0453, -0.035)
O intervalo para a diferença das proporções da variável 'diarreia' é entre (-0.0389, -0.0266)
O intervalo para a diferença das proporções da variável 'descon_resp' é entre (0.1012, 0.1214)
O intervalo para a diferença das proporções da variável 'dispneia' é entre (0.1336, 0.1512)
O intervalo para a diferença das proporções da variável 'dor_de_garganta' é entre (-0.1075, -0.0937)
O intervalo para a diferença das p

O intervalo da diferença é bem pequeno, isso se deve pelo tamanho de cada um dos grupos, o que torna a diferença bem relevante. Então nossas constatações não diferem muito: a saturação de oxigênio abaixo de 95 é a maior diferença entre os grupos, em seguida tem a dispnéia e por fim o desconforto respiratório.

Agora, façamos uma análise semelhante em relação às comorbidades. O primeiro passo é também criar subgrupos, onde a coluna 'comorbidades' não está nula.

In [ ]:
recuperados_comorbidades = recuperados[recuperados['comorbidades'].notna()].iloc[:,-9:]
obitos_comorbidades = obitos[obitos['comorbidades'].notna()].iloc[:,-9:]

In [ ]:
recuperados_comorbidades.shape, obitos_comorbidades.shape

((135495, 9), (15198, 9))

In [ ]:
recuperados_comorbidades.mean()

diabetes           0.284195
card_or_vasc       0.053840
hepatic_cronic     0.001173
renais_cronic      0.029418
resp_cronic        0.163430
gestacao           0.001476
imunossupressao    0.047419
cromossomicas      0.016340
obesidade          0.012842
dtype: float64

In [ ]:
comparacao_comorb = pd.DataFrame({'obitos':obitos_comorbidades.mean(),'recuperados':recuperados_comorbidades.mean()})
comparacao_comorb['diferenca'] = comparacao_comorb['obitos'] - comparacao_comorb['recuperados']
print("Diferença na proporção de comorbidades no grupo de óbitos e no de recuperados\n")
print(comparacao_comorb.sort_values(by = 'diferenca', ascending = False))

Diferença na proporção de comorbidades no grupo de óbitos e no de recuperados

                   obitos  recuperados  diferenca
card_or_vasc     0.691736     0.053840   0.637896
diabetes         0.495263     0.284195   0.211068
obesidade        0.125740     0.012842   0.112898
renais_cronic    0.096394     0.029418   0.066976
hepatic_cronic   0.025530     0.001173   0.024356
gestacao         0.000658     0.001476  -0.000818
cromossomicas    0.002698     0.016340  -0.013642
imunossupressao  0.027175     0.047419  -0.020244
resp_cronic      0.104027     0.163430  -0.059404


Quando olhamos para diferença das proporções de comorbidades, os maiores valores são:



*   Doenças cardíacas ou vasculares (63,79%)
*   Diabetes (21,11%)
*   Obesidade/Sobrepeso (11,29%)
*   Doenças renais crônicas (6,70%)

Surpreendemente, não houve grande diferença na proporção de "doenças respiratórias crônicas", na verdade, o resultado foi até negativo, sugerindo que a proporção dessa comorbidade é maior naqueles que sobreviveram ao COVID-19.

Realizando o teste para igualdade de proporções das comorbidades, temos novamente as hipóteses:

*   H0: (Prop.O - Prop.R) <= 0 (Proporção em 'obitos' menos Proporção em 'recuperados' menor ou igual a zero)
*   H1: (Prop.O - Prop.R) > 0 (Proporção em 'obitos' menos Proporção em 'recuperados' maior que zero)

Para um teste unilateral à direita com a = 0,01, temos que o Zcrítico é de 2,327. Então nossa regra de decisão:

*  Rejeitamos H0  se Zcalc > 2,327
*  Caso contrário, não rejeitar H0


Utilizando da função que já foi criada temos que:

In [ ]:
colunas = recuperados_comorbidades.columns
for col in colunas:
  teste_prop = proportions_test(obitos_comorbidades,recuperados_comorbidades,col)
  print("O teste para diferença das proporções da coluna '{}' foi de {}".format(col,round(teste_prop,2)))

O teste para diferença das proporções da coluna 'diabetes' foi de 53.57
O teste para diferença das proporções da coluna 'card_or_vasc' foi de 231.0
O teste para diferença das proporções da coluna 'hepatic_cronic' foi de 47.34
O teste para diferença das proporções da coluna 'renais_cronic' foi de 41.93
O teste para diferença das proporções da coluna 'resp_cronic' foi de -19.07
O teste para diferença das proporções da coluna 'gestacao' foi de -2.56
O teste para diferença das proporções da coluna 'imunossupressao' foi de -11.37
O teste para diferença das proporções da coluna 'cromossomicas' foi de -13.14
O teste para diferença das proporções da coluna 'obesidade' foi de 85.83


No caso das comorbidades, 5 das 9  tiveram uma estatística de teste maior que a pré-definida, sugerindo que a proporção dessas comorbidades realmente é diferente entre os grupos, realizando um teste para encontrarmos o intervalo em que estaria essa diferença:

In [ ]:
colunas = recuperados_comorbidades.columns
for col in colunas:
  teste_prop = intervalo_diferenca(obitos_comorbidades,recuperados_comorbidades,col)
  print("O intervalo para a diferença das proporções da variável '{}' é entre {}".format(col,teste_prop))

O intervalo para a diferença das proporções da variável 'diabetes' é entre (0.2002, 0.222)
O intervalo para a diferença das proporções da variável 'card_or_vasc' é entre (0.6281, 0.6477)
O intervalo para a diferença das proporções da variável 'hepatic_cronic' é entre (0.0211, 0.0277)
O intervalo para a diferença das proporções da variável 'renais_cronic' é entre (0.0607, 0.0733)
O intervalo para a diferença das proporções da variável 'resp_cronic' é entre (-0.0663, -0.0525)
O intervalo para a diferença das proporções da variável 'gestacao' é entre (-0.0014, -0.0002)
O intervalo para a diferença das proporções da variável 'imunossupressao' é entre (-0.024, -0.0165)
O intervalo para a diferença das proporções da variável 'cromossomicas' é entre (-0.015, -0.0122)
O intervalo para a diferença das proporções da variável 'obesidade' é entre (0.1059, 0.1199)


#CONCLUSÃO

Com os resultados encontrados até então, há a constatação de uma aparente diferença na presença de alguns aspectos entrem aqueles que não resistem ao COVID-19 e aqueles que conseguem se recuperar, como resultado desse estudo conclui-se que:


* Em relação ao sexo, os casos dos homens foram mais fatais que os das mulheres
* Como já era esperado, pessoas mais velhas são as que mais morrem.
* Em relação aos sintomas, baixa saturação de oxigênio, dispnéia e desconforto respiratório estão proporcionalmente mais presentes nos casos mais fatais.
* Ja a febre, dor de garganta, cefaleia e alteração de olfato/paladar estão proporcionalmente mais presentes nos casos em que as pessoas conseguem se recuperar.

* Quando olhamos as comorbidades, as doenças cardíacas, vasculares, diabetes, doenças renais e hepáticas estão proporcionalmente mais presentes nos casos fatais.
* Doenças respiratórias crônicas, doenças imunossupressoras e até mesmo a gestação são comorbidades proporcionalmente mais presentes nos casos em que as pessoas conseguem se recuperar.

